In [1]:
using WordNet
using WordEmbeddings, SoftmaxClassifier
using Utils
using Query
using Distances
using Iterators

  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31


In [152]:
const WN_PATH="data/corpora/WordNet-2.1/"
db = DB(WN_PATH)

"data/corpora/WordNet-2.1/"

In [4]:
ee = restore("models/ss/tokenised_lowercase_WestburyLab.wikicorp.201004_100_i1.model");

In [5]:
function normal_probs(logprobs::Vector)
    ret = copy(logprobs)
    max_lp = maximum(logprobs)
    ret.-=max_lp #Bring closer to zero
    map!(exp,ret)
    denom = sum(ret)
    ret./=denom
    ret
end
function weighted_average(logprobs, embeddings)
    ret = zeros(first(embeddings))
    for (weight, embedding) in zip(normal_probs(logprobs), embeddings)
        ret.+= weight.*embedding
    end
    ret
end

function all_logprobs_of_context(embed, context, middle_word)
    [Query.logprob_of_context(embed, context,wv; skip_oov=true, normalise_over_length=true)
        for wv in embed.embedding[middle_word]]
end

function synthesize_embedding(ee,context, middle_word)
    logprobs = all_logprobs_of_context(ee, context,middle_word)
    weighted_average(logprobs, ee.embedding[middle_word])
end

synthesize_embedding (generic function with 1 method)

  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
in display_dict at /home/ubuntu/.julia/v0.5/IJulia/src/execute_request.jl


In [6]:
function poormans_tokenize(source)
    cleaned = filter(s->!ispunct(s), lowercase(source))
    map(string,  split(cleaned))::Vector{SubString{String}}
end

function punctuation_space_tokenize(source)
    preprocced = lowercase(source)
    pass1=replace(preprocced,r"[[:punct:]]*[[:space:]][[:punct:]]*"," ")
    pass2=replace(pass1,r"[[:punct:]]*$|^[[:punct:]]*","")
    split(pass2)
end



punctuation_space_tokenize (generic function with 1 method)

In [7]:
function lexical_informed_embeddings(db,ee, word)
    source_wvs = ee.embedding[word]
    target_synsets = Base.flatten((haskey(db.lemmas[pos], word) ? synsets(db, db[pos, word]) : WordNet.Synset[] 
        for pos in keys(WordNet.SYNSET_TYPES))) 

    if length(source_wvs) == 1
        #Use MCWS
        lemma = db['n', lemma_word]
        target_synsets = synsets(db, lemma);
        score, index = findmax((get(ss.word_counts,lemma_word,0)+0.1sum(values(ss.word_counts))) for ss in target_synsets)
        target_synsets=[target_synsets[index]] # Throw out others
    else
            
    synsets4vectors = Dict{eltype(source_wvs), Synset}()
    for synset::Synset in target_synsets
            context::Vector{SubString{String}} = collect(filter!(w->w!=word, punctuation_space_tokenize(synset.gloss)))
            wv = synthesize_embedding(ee,context, word)
            synsets4vectors[wv] = synset
    end
    synsets4vectors
end

lexical_informed_embeddings (generic function with 1 method)

In [46]:
@time lexical_informed_embeddings(db,ee, "federal")

  0.024479 seconds (102.73 k allocations: 1.726 MB)


Dict{Array{Float32,1},WordNet.Synset} with 6 entries:
  Float32[-0.0537537,0.0682679,-0.0259331,0.157535,-0.272935,0.186007,-0.0694081,-0.0785202,0.199033,-0.126381,-0.0847547,0.128801,-0.183493,-0.0422632,0.0892356,-0.22934,-0.0623469,-0.00149297,0.056994,0.0964768,0.0161398,0.0781163,0.211676,-0.0763543,0.144749,0.216146,0.0675393,0.088937,-0.118281,-0.356966,0.0163716,-0.0818849,0.0128478,-0.0469062,-0.0810498,0.203175,0.0173182,0.0536753,0.0529723,-0.0489318,0.0774052,-0.0222965,-0.0963653,0.0107006,0.0746002,-0.082665,-0.199656,-0.186337,-0.0727619,-0.0921054,0.103729,0.0355965,0.0991122,0.114297,0.0437884,0.0641925,-0.155373,-0.191694,0.106501,0.0299754,0.202594,0.0104121,0.00473919,-0.049686,0.203001,0.149391,-0.0946662,-0.00488328,0.0074259,-0.189963,0.206126,0.0392408,-0.0732178,-0.0386577,0.0215429,0.181814,0.00604329,0.0138382,-0.0173619,-0.0353349,-0.0625751,-0.151501,0.00638058,0.139065,0.130912,0.0440749,0.156332,0.108061,0.0793152,0.0144482,0.140982,0.0163569,-0.061246,0

In [106]:
l1 = db['a',"six"]
l1.

1-element Array{AbstractString,1}:
 "&"

In [ ]:


function get_sense_key(synset, lemma):
    index = 0
    try: #try and do better
        index = synset.lemma_names().index(lemma)
    except ValueError:
        pass #couldn't do better
    return synset.lemmas()[index].key()

def eval_wsd(wsd, text_id, tagged_phrases, indexed_instances):
    lemmas_and_synsets = wsd(tagged_phrases)
    for ii, id in indexed_instances.items():
        #print(id)
        lemma, synset = lemmas_and_synsets[ii]
        comment = "!! lemma="+''.join([i if ord(i) < 128 else ' ' for i in lemma])
        if synset: #If it was even defined in wordnet
            sensekey = get_sense_key(synset,lemma)
            yield text_id, id, sensekey, comment
        else:
            yield text_id, id, comment
        

In [148]:

function print_structure(v)
    for fn in fieldnames(v)
        println(string(fn),"\t", getfield(v,fn))
    end   
end

l1 = db['v',"discover"]
sss = synsets(db,l1)

print_structure(l1)
println("-"^10)
#d001 d001.s001.t006 six%5:00:00:cardinal:00 !! lemma=six

word	discover
pos	v
tagsense_count	7
synset_offsets	[2134693,1623484,591374,713143,2265142,924278,2108435,644350]
id	3141
pointer_syms	AbstractString["@","~","*",">","\$","+"]
----------


In [173]:
[map(bin,".22-calibre%3:01:00::"|> collect).== map(bin,".22-caliber%3:01:00::"|> collect)]

1-element Array{BitArray{1},1}:
 Bool[true,true,true,true,true,true,true,true,true,false,false,true,true,true,true,true,true,true,true,true,true]

  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
in display_dict at /home/ubuntu/.julia/v0.5/IJulia/src/execute_request.jl


In [ ]:

function load_sense_key_map(wn_basedir)
    path=joinpath(wn_basedir, "dict","index.sense")
    sense_key_map = Dict{Tuple{Int64,String},String}()
    for line in eachline(path)
        full_sense_key, offset_str, sense_num_str, tagcount_str = split(line) 
        lemma_name = first(split(full_sense_key,'%'))
        sense_offset = parse(Int64, offset_str)
        index = (sense_offset,lemma_name)
        @assert(!haskey(sense_key_map, index))
        sense_key_map[index] = full_sense_key
    end
    sense_key_map
end


function sense_key(lem::Lemma, synset::Synset,sense_key_map)
    sense_key_map[(s1.offset,lem.word)]
end
function sense_keys(lem::Lemma,sense_key_map)
    [sense_key_map[(ss_offset,lem.word)] for ss_offset in lem.synset_offsets ]
end

In [181]:
sense_key_map = load_sense_key_map(WN_PATH)

sense_keys (generic function with 1 method)

In [184]:
sense_key(l1,s1, sense_key_map)
sense_keys(db['a',"six"],sense_key_map)

1-element Array{String,1}:
 "six%5:00:00:cardinal:00"

In [176]:
sense_key_map[2134693,"discover"]

"discover%2:39:03::"

In [149]:
for (ii,si) in enumerate(sss)
    println(ii)
    print_structure(si)
    println("-"^10)
end



1
offset	2134693
lex_filenum	39
word_counts	Dict{AbstractString,Int64}(Pair{AbstractString,Int64}("discover",3),Pair{AbstractString,Int64}("find",2),Pair{AbstractString,Int64}("observe",2),Pair{AbstractString,Int64}("notice",4),Pair{AbstractString,Int64}("detect",0))
synset_type	v
pos	v
pointers	WordNet.Pointer[WordNet.Pointer("\$",0x0018c5bc,'v',"00","00"),WordNet.Pointer("@",0x0020b735,'v',"00","00"),WordNet.Pointer("+",0x0055f43b,'n',"05","01"),WordNet.Pointer("+",0x0097addf,'n',"04","02"),WordNet.Pointer("+",0x006c9310,'n',"04","01"),WordNet.Pointer("+",0x0057884e,'n',"04","01"),WordNet.Pointer("+",0x0000a24c,'n',"04","01"),WordNet.Pointer("+",0x0000a24c,'n',"03","02"),WordNet.Pointer("+",0x0097addf,'n',"03","01"),WordNet.Pointer("+",0x008b90ec,'n',"03","01"),WordNet.Pointer("+",0x00024479,'n',"03","02"),WordNet.Pointer("+",0x00146188,'a',"02","03"),WordNet.Pointer("+",0x001b98b0,'a',"02","01"),WordNet.Pointer("+",0x0090cc14,'n',"02","03"),WordNet.Pointer("+",0x0065df24,'n',"02","0

In [102]:
#See http://webcache.googleusercontent.com/search?q=cache:b5Ixl2Ds2zcJ:icl.pku.edu.cn/member/yujs/papers/pdf/corpus-format.pdf+&cd=12&hl=en&ct=clnk&gl=au&client=firefox-b-ab
#See http://moin.delph-in.net/SemCor
function sense_id_format(db::DB, lem::Lemma)
    #lemma%ss_type:lex_filenum:lex_id)
    ss = Synset(db, lem)
    ss_type =  find(['n','v','a','r','s'], ss.pos) |> string
    lex_filenum = lpad(lem.lex_filenum,2,'0')

search: getfield



```
getfield(value, name::Symbol)
```

Extract a named field from a `value` of composite type. The syntax `a.b` calls `getfield(a, :b)`.


In [ ]:
using LightXML

In [ ]:
immutable wsd_challenge
    id
    word
    lemma
    pos
    context
end

function load_challenges(xml_file="data/corpora/wsd/semeval2007_t7/test/eng-coarse-all-words.xml")
    xdoc = parse_file(xml_file)
    xroot = root(xdoc)
    Task() do
        for text_node in child_elements(xroot)
            #@show text_node
            #text = child_nodes(text_node) |> collect
            #println(text)
            for sentence_node in child_elements(text_node)
                sentence = punctuation_space_tokenize(content(sentence_node))

                for lemma_node in child_elements(sentence_node)
                    word = content(lemma_node) |> lowercase
                    lemma = attribute(lemma_node,"lemma")|> lowercase
                    pos = attribute(lemma_node,"pos")
                    id = attribute(lemma_node,"id")
                    context = filter(w->w!=word, sentence)
                    produce(wsd_challenge(id, word, lemma, pos, context))
                end
            end
        end
    end
end
challenges = load_challenges() |> collect